In [1]:
import pandas as pd
import time
import os
import pandas_gbq
from google.cloud import bigquery
import numpy as np
import re

In [2]:
os.chdir('G:\\Drives compartilhados\\República.org\\4. Equipes\\Dados e Comunicação\\DADOS\\415 - Repositório de Dados\\Dados para subir\\Dados Brutos\\Atlas do Estado')


In [3]:
df = pd.read_csv('SIAPE_ativos_economia_rural.csv')
df

,ano,ministerios,servidores_ativos_ministerios,total_servidores_ativos,perc_servidores
0,1999,"ministerio da agricultura, pecuaria e abasteci...",11036,482854,2.29
1,1999,ministerio da defesa,32124,482854,6.65
2,1999,ministerio das relacoes exteriores,3193,482854,0.66
3,1999,ministerio do esporte,208,482854,0.04
4,1999,ministerio do turismo,224,482854,0.05
...,...,...,...,...,...
118,2018,"ministerio da agricultura, pecuaria e abasteci...",9893,611916,1.62
119,2018,ministerio da defesa,17771,611916,2.90
120,2018,ministerio das relacoes exteriores,3337,611916,0.55
121,2018,ministerio do esporte,330,611916,0.05


In [4]:
df['ministerios'] = df['ministerios'].str.title() #padronizando

In [5]:
def transformar(nome):
    nome = re.sub(r"\bMinisterio\b", "Ministério",nome)
    nome = re.sub(r"\bPecuaria\b", "Pecuária",nome)
    nome = re.sub(r"\bRelacoes\b", "Relações",nome)
    nome = re.sub(r"\bAgrario\b", "Agrário",nome)
    return nome

In [6]:
df['ministerios'] = df['ministerios'].apply(transformar)

In [7]:
df['ministerios'].unique()


array(['Ministério Da Agricultura, Pecuária E Abastecimento',
       'Ministério Da Defesa', 'Ministério Das Relações Exteriores',
       'Ministério Do Esporte', 'Ministério Do Turismo',
       'Ministério Do Desenvolvimento Agrário',
       'Ministério De Pesca E Aquicultura'], dtype=object)

In [8]:
df = df.drop(['total_servidores_ativos'], axis=1)

In [9]:
df

,ano,ministerios,servidores_ativos_ministerios,perc_servidores
0,1999,"Ministério Da Agricultura, Pecuária E Abasteci...",11036,2.29
1,1999,Ministério Da Defesa,32124,6.65
2,1999,Ministério Das Relações Exteriores,3193,0.66
3,1999,Ministério Do Esporte,208,0.04
4,1999,Ministério Do Turismo,224,0.05
...,...,...,...,...
118,2018,"Ministério Da Agricultura, Pecuária E Abasteci...",9893,1.62
119,2018,Ministério Da Defesa,17771,2.90
120,2018,Ministério Das Relações Exteriores,3337,0.55
121,2018,Ministério Do Esporte,330,0.05


In [10]:
df.rename(columns={'perc_servidores': 'per_servidores'}, inplace=True)
df

,ano,ministerios,servidores_ativos_ministerios,per_servidores
0,1999,"Ministério Da Agricultura, Pecuária E Abasteci...",11036,2.29
1,1999,Ministério Da Defesa,32124,6.65
2,1999,Ministério Das Relações Exteriores,3193,0.66
3,1999,Ministério Do Esporte,208,0.04
4,1999,Ministério Do Turismo,224,0.05
...,...,...,...,...
118,2018,"Ministério Da Agricultura, Pecuária E Abasteci...",9893,1.62
119,2018,Ministério Da Defesa,17771,2.90
120,2018,Ministério Das Relações Exteriores,3337,0.55
121,2018,Ministério Do Esporte,330,0.05


In [11]:
df['per_servidores'] = df['per_servidores']/100

In [12]:
df

,ano,ministerios,servidores_ativos_ministerios,per_servidores
0,1999,"Ministério Da Agricultura, Pecuária E Abasteci...",11036,0.0229
1,1999,Ministério Da Defesa,32124,0.0665
2,1999,Ministério Das Relações Exteriores,3193,0.0066
3,1999,Ministério Do Esporte,208,0.0004
4,1999,Ministério Do Turismo,224,0.0005
...,...,...,...,...
118,2018,"Ministério Da Agricultura, Pecuária E Abasteci...",9893,0.0162
119,2018,Ministério Da Defesa,17771,0.0290
120,2018,Ministério Das Relações Exteriores,3337,0.0055
121,2018,Ministério Do Esporte,330,0.0005


Subindo para o GBQ

In [13]:
client = bigquery.Client()
dataset_ref = client.dataset('perfil_remuneracao')

c:\Users\ana.sales_republica\Anaconda\lib\site-packages\google\auth\_default.py:78: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [14]:
schema= [bigquery.SchemaField('ano','INTEGER',description='Ano de referência da observação'),
          bigquery.SchemaField('ministerios','STRING',description='Ministérios observados'),
          bigquery.SchemaField('servidores_ativos_ministerios', 'INTEGER',description='Número de servidores ativos por ministério'),
          bigquery.SchemaField('per_servidores','FLOAT', description='Percentual de servidores ativos por ministério ')]

In [15]:
table_ref = dataset_ref.table('SIAPE_ativos_economia_rural') 
job_config = bigquery.LoadJobConfig(schema=schema)
job = client.load_table_from_dataframe(df, table_ref, job_config=job_config)
job.result()

LoadJob<project=repositoriodedadosgpsp, location=US, id=17beebdb-db1b-42b0-87a7-c4cca52fa129>